In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#필요 패키지 설치
!pip install numpy==1.23.1
!pip install mxnet
!pip install gluonnlp
!pip install pandas tqdm
!pip install sentencepiece==0.1.96
!pip install transformers==4.8.2
!pip install torch==1.8.1

  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661751 sha256=5ff048b9897726a78a285def5ebb6fa7ebacce5456e153df64d5001990cad8ec
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#KoBERT 깃허브에서 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6xtot8lj/kobert-tokenizer_9d7fd9824163440d8b049ce0780876bb
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6xtot8lj/kobert-tokenizer_9d7fd9824163440d8b049ce0780876bb
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=5cc69f0a5cc45b07e3e4fe2d732b9707f70e59d0b3a8909196ece3700fd3eb4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ey6doftk/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-nmk6q259
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-nmk6q

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import pandas as pd

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
device = torch.device("cuda:0")
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [6]:
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋
chatbot_data  = pd.read_csv("/content/drive/MyDrive/dataset/5차년도_2차.csv", encoding='cp949')

In [9]:
# "상황" 사용하기
chatbot_data.loc[(chatbot_data['상황'] == "fear"), '상황'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['상황'] == "angry"), '상황'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['상황'] == "sadness"), '상황'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['상황'] == "neutral"), '상황'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['상황'] == "happiness"), '상황'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['상황'] == "disgust"), '상황'] = 6  #혐오 => 6

In [7]:
# 7개의 감정 class → 숫자
chatbot_data.loc[(chatbot_data['emotion'] == "fear"), 'emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['emotion'] == "surprise"), 'emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['emotion'] == "angry"), 'emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['emotion'] == "sadness"), 'emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['emotion'] == "neutral"), 'emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['emotion'] == "happiness"), 'emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['emotion'] == "disgust"), 'emotion'] = 6  #혐오 => 6

KeyError: 'emotion'

In [11]:

# chatbot_data.sample(n=10)
# # 7개의 감정 class → 숫자
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "공포"), 'Unnamed: 2'] = 0  #공포 => 0
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "놀람"), 'Unnamed: 2'] = 1  #놀람 => 1
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "분노"), 'Unnamed: 2'] = 2  #분노 => 2
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "슬픔"), 'Unnamed: 2'] = 3  #슬픔 => 3
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "중립"), 'Unnamed: 2'] = 4  #중립 => 4
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "행복"), 'Unnamed: 2'] = 5  #행복 => 5
# chatbot_data.loc[(chatbot_data['Unnamed: 2'] == "혐오"), 'Unnamed: 2'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['발화문'], chatbot_data['상황'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data)
print(data_list[:10])

# chatbot_data.sample(n=10)
# # 7개의 감정 class → 숫자
# chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
# chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
# chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
# chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
# chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
# chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
# chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

# data_list = []
# for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
#     data = []
#     data.append(q)
#     data.append(str(label))

#     data_list.append(data)

# print(data)
# print(data_list[:10])

['걱정해줘서 고마워.', '6']
[['헐! 나 이벤트에 당첨 됐어.', '5'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '5'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '5'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '5'], ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '4'], ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '4'], ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '5'], ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '5'], ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '5'], ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '5']]


In [12]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)


In [13]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [15]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
tok=tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-22-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch 1 batch id 1 loss 2.022535562515259 train acc 0.140625
epoch 1 batch id 201 loss 0.5260992646217346 train acc 0.6075870646766169


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

epoch 4 test acc 0.9243559718969555


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.013563944958150387 train acc 1.0
epoch 5 batch id 201 loss 0.07949571311473846 train acc 0.9780783582089553
epoch 5 train acc 0.9774948559670782


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9233313817330211


In [23]:
state = {
    'state_dict': model.state_dict(),
}

torch.save(state, "/content/drive/MyDrive/dataset/bert_classifier_model_v2.pth")

In [29]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 4:
                test_eval.append("중립")
            elif np.argmax(logits) == 5:
                test_eval.append("행복")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오")

        print(test_eval[0], "???")

In [30]:
inf_data  = pd.read_excel("/content/drive/MyDrive/dataset/tesla.xlsx")
inf_data = inf_data.drop(0)

In [31]:
inf_data = inf_data[["text", "emotion"]]
inf_data

,text,emotion
1,오늘 하루도 힘찬 하루되세요. 우리가 테슬라 믿어주지 누가 믿어줍니까? ㅋ,기쁨
2,"테슬라의 주가가 쭉쭉 떨어지자 테슬라에 대한 정보를 다루는 유튜브, ‘오늘의 테슬라...",기쁨
3,"아내가 동네 아주머니들 한분이 생일이라 점심 모임 다녀왔는데, 화제 중 일부가 FS...",기쁨
4,"테슬라는 현재 사실상 FSD와 로보택시 올인 모드고, 데이터센터 구축에 엄청난 돈을...",공포
5,테슬라 해고 인원 해고 사실 몰라 출근 후 사원증 통하지 않은 사실 알고 집에 돌아...,혐오
...,...,...
66,테슬라 모델2. 가만히 보니 아이폰의 se느낌나네요,중립
67,머스크가 트위터에 시간을 많이 쓰니 테슬라 하락에 풀베팅을 하심이,혐오
68,현명한 와이프를 두셨네요. 현명한 하이프를 고른 안목처럼 테슬라 또한 실망시키지 않...,기쁨
69,조금만 참으세요 $tsla 주주님들~ 일론머스크 형이 있잖아요~ 테슬라 개인주주들이...,기쁨


In [32]:
for index, row in inf_data.iterrows():
    text = row['text']
    emotion = row['emotion']
    predict(text)
    print(text)
    print(emotion)

    print("-------------------")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


슬픔 ???
오늘 하루도 힘찬 하루되세요. 우리가 테슬라 믿어주지 누가 믿어줍니까? ㅋ
기쁨
-------------------
슬픔 ???
테슬라의 주가가 쭉쭉 떨어지자 테슬라에 대한 정보를 다루는 유튜브, ‘오늘의 테슬라 뉴스’에 조롱의 댓글들이 많이 보이기 시작했다. 테슬라에 대한 조롱, 비난글이 점차 늘어나다가 어느 순간 무관심에 그칠 때가 바닥이었고 점점 그때가 다가오고 있음을 알 수 있다. 바닥을 잡는 것은 신의…
기쁨
-------------------
행복 ???
아내가 동네 아주머니들 한분이 생일이라 점심 모임 다녀왔는데, 화제 중 일부가 FSD. 대부분 테슬라 모시거나 몰았었기 때문에 이야기가 잘 통하는 듯. 
기쁨
-------------------
슬픔 ???
테슬라는 현재 사실상 FSD와 로보택시 올인 모드고, 데이터센터 구축에 엄청난 돈을 쓰고 있음. 데이터센터 구축 비용을 한꺼번에 많이 쓰면 실적이박살난다.
공포
-------------------
분노 ???
테슬라 해고 인원 해고 사실 몰라 출근 후 사원증 통하지 않은 사실 알고 집에 돌아가 X때도 그랬는데 테슬라도 똑같네 잔인한거 같긴하다
혐오
-------------------
분노 ???
미국에 소비자 들은 중고차 가격에 민감 한데 지난 몇년을 테슬라 차량을 인하 하다 보니 계속해서 똥 값이 되는 차량을 사야 할 이유가 없고. 렌트카 회사들은 대량으로 사면서 활인을 많이 받고 그걸 나중에 재 판매를 해서 돈을 버는데 차 값이 계속 떨어지는 테슬라 차량은 악몽 그 자체다. 
공포
-------------------
슬픔 ???
테슬라 공매도 잔량 업데이트 (3/28 기준) : 107,481,521주 (ATH 갱신)
공포
-------------------
슬픔 ???
지금 시점 테슬라 주주 99%가 손해
슬픔
-------------------
슬픔 ???
테슬라 대차게 까는 쭈니형...ㅋㅋㅋ 우리 테슬람 재형이...안쓰러움..
슬픔
-------------------
슬픔 ?